## Module: Getting Started with Spark's RDD

- Understanding the role of spark in data analysis

- Understanding RDD and their characteristics

- Installing Spark Standalone in a local environment

- Loading data from a file

- Reading data from an RDD

In Data Science, 80% of the time spent prepare data, 20% of time spent complain about need for prepare data.

### Data Processing Tasks

- Parsing fields from text
- Accounting for missing values
- Identifying and investigating anomalies
- Summarizing using tables and charts

### Data Complexity

#### Small Data (somewhat Messy)

- Low data collection frequency
- 10s-1000s of rows per day
- sometimes involves manual data collection
- This kind of data can be handled by Spreadsheets

#### Medium Data (High Data Integrity)

- High frequency of collection
- 100k rows per day
- Programmatically collected
- ACID properties
- Handled by Databases using tool like sql

#### Big Data (very messy)

- very high frequency of data collection
- Millions/Billions of rows per day
- Files stored across a cluster of machines
- Many many files (webpages, log files)
- Handled by Distributed Computing engine like Hadop, MapReduce, Spark


## Here we are going to focus on Handling Big Data by using Apache Spark using Python.

### Spark

- An engine for data procrssing and analysis. It is General Purpose, Interactive and uses Distributed Computing engine.

#### Why General Purpose?

- Exploring
- Cleaning and Preparing
- Applying Machine Learning
- Building Data Applications

#### Why Interactive?

- REPL: Read-Evaluate-Print-Loop
- Interactive environments Fast feedback

#### Why Distributed Computing?

- Process data across a cluster of machines
- Integrate with Hadoop




## Spark APIs: Scala, Python and Java

Here we will mostly focus on Spark Python API.

Almost all data is processed using Resilient Distributed Datasets.

- RDDs are the main programming abstraction in spark
- RDDS are in-memory collections of objects
- With RDDs, you can interact and play with billions of row of data

### Spark is made up of a few different components:

- Spark Core: The basic functionality of spark RDDs. Spark Core is just a computing engine. It needs two additional components that are A Storage System that stores the data to be processed and A cluster Manager to help Spark run tasks across a cluster of machines. Both of these are plug and play components.

For Storage Sytem we can choose Local file System in standalone mode or we can connect it with HDFS if integrating it with Hadoop.

For Cluster Manager we can choose Built-in cluster Manager comes in a standalone mode or we could plug in YARN if integrating with Hadoop.

## Installing Spark

### Prerequisites

- Java 7 or above
- Scala
- IPython (Anaconda) then

- Go and download Spark binaries
- Update environment variables

#### Spark Environment Variables

- SPARK_HOME: Point to the folder where Spark has been extracted

- PATH: %SPARK_HOME%/bin

#### Configure iPython Notebook for Spark

For this we have to set two more environment variable

- PYSPARK_DRIVER_PYTHON: ipython

- PYSPARK_DRIVER_PYTHON_OPTS: "notebook"






## RDD: Resilient Distributed Datasets

- Collection of records which are kept in memory
- Primary way by which we integrate data in spark
- RDDs have some special characteristics: 
 -  Partitions, 
 -  Read-only, 
 -  Lineage.

#### Partitions
- RDDs represent data in-memory
- Data is divided into partitions
- Distributed to multiple machines, called nodes
- Nodes are individual machine in cluster
- Nodes process data in parallel

#### Read-Only
- RDDs are immutabel
- Only two types of operations
 - Transformation: Transform into another RDD
 - Action: Request a result

 #### Transformation
 - A data set loaded inot an RDD
 - The user may define a chain of transformations on the dataset
 - Load data, Pick only the 3rd column, Sort the values
 - Wait until a result is requested before executing any of these transformations till then any RDD only contain metadata.

 #### Action
 - Request a result using an action
 - Such as The first 10 rows, A count, A sum
 - Data is processed only when the user requests a result
 - The chain of tranformations define earlierr is executed
 - Spark keeps a record of the series of transformations requested by the user
 - When the action is called upon, it groups the transformations in an efficient way when an Action is requested
 - It is Lazy Evaluation of Spark

#### Lineage
- When created, an RDD just holds metadata: A tranformation and its parent RDD
- Every RDD knows where it came from
- Lineage can be tarced back all the way to the source
- When an action is requested on an RDD
- All its parent RDDs are materialized
- The characteristic of lineage is what allows RDDs to have resilience and Lazy Evaluation
- Resilience: In-built fault tolerance, if something goes wrong, reconstruct from source
- Lazy Evaluation: Materialize only when necessary








### Module: Transforming and Cleaning Unstructured Data

- Transforming data using functional constructs i.e. filter, map and reduce
- Cleaning unstructured data
- Identifying and removing anomalies, missing values

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 38.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=80b04bfa6f20c42d5bc75b5ee14461db43afb29e164a35b4e0fde5becf1f1b6e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NYCrimeAnalysis").getOrCreate()
sc = spark.sparkContext

In [3]:
# load the data and get a quick sense
path = "/content/drive/MyDrive/SAS2PYS/data/NYPD_7_Major_Felony_Incidents.csv"
data = sc.textFile(path)

In [4]:
data.take(10)

['OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1',
 '1,f070032d,09/06/1940 07:30:00 PM,Friday,Sep,6,1940,19,9,7,2010,BURGLARY,FELONY,D,66,BROOKLYN,N.Y. POLICE DEPT,987478,166141,"(40.6227027620001, -73.9883732929999)"',
 '2,c6245d4d,12/14/1968 12:20:00 AM,Saturday,Dec,14,1968,0,12,14,2008,GRAND LARCENY,FELONY,G,28,MANHATTAN,N.Y. POLICE DEPT,996470,232106,"(40.8037530600001, -73.955861904)"',
 '3,716dbc6f,10/30/1970 03:30:00 PM,Friday,Oct,30,1970,15,10,31,2008,BURGLARY,FELONY,H,84,BROOKLYN,N.Y. POLICE DEPT,986508,190249,"(40.688874254, -73.9918594329999)"',
 '4,638cd7b7,07/18/1972 11:00:00 PM,Tuesday,Jul,18,1972,23,7,19,2012,GRAND LARCENY OF MOTOR VEHICLE,FELONY,F,73,BROOKLYN,N.Y. POLICE DEPT,1005876,182440,"(40.6674141890001, -73.9220463899999)"',
 '5,6e410287,05/21/1987 12:01:00

### Transforming Data with Spark

- To transform data in Spark we use a special paradigm called the functional paradigm called the functional paradigm.
- As we know, RDD such is a collection of records
- Lists and maps in python, all data frames in R might be collections that we have come across before. Any transformation or computation on this collection of objects involve doing something with each item in the collection.
- One way of doing this is Imperative way using loops
- On the other haand we could use Functional way.
- The functional way will perform an operation independently on every element of the record at the same time and return a new set of records. So it doesn't modify each record in place. It will return a new set of records.
- Apply the same function to each record
- Functional programming allows us to process data in parallel, which is why it fits in really well with a distributed computing system like spark.
- Spark uses the functional programming way to actually perform operations on RDDs.
- The function can be Explicitly Defined Functions or Lambda Function.




### Functional Programming

It basically involves taking a function and applying it on each record of a collection. In spark those collection are RDDs.
- Filter
 - The function can be used to filter records which match a certain condition.
 - The result of the filter operation would be a new RDD in which we have dropped all the records that don't match the condition that we have specified in our boolean function.
- Map 
 - They can be used to map or transform each record to a new record.
- Reduce 
 - They can be used to combine the records in a specified way. For instance recompute a sum.
 - The reduce operation is slightly different from the filter and map operations, Which are truly applied in parallel on all records in the RDD. The reduce operation on the other hand is applied on two records at a time.
 - Unlike the filter and map operations which take in functions with a single argument, the argument representing one record but the reduce operation takes in a function with two arguments.


### Filter

In [5]:
# Filtering the header row
header = data.first()
print(header)

OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1


In [6]:
dataWoHeader = data.filter(lambda x: x != header)

In [7]:
dataWoHeader.first()

'1,f070032d,09/06/1940 07:30:00 PM,Friday,Sep,6,1940,19,9,7,2010,BURGLARY,FELONY,D,66,BROOKLYN,N.Y. POLICE DEPT,987478,166141,"(40.6227027620001, -73.9883732929999)"'

### Transforming records from string to named tuples

In [8]:
# Parse the rows to extract fields
dataWoHeader.map(lambda x:x.split(",")).take(10)

[['1',
  'f070032d',
  '09/06/1940 07:30:00 PM',
  'Friday',
  'Sep',
  '6',
  '1940',
  '19',
  '9',
  '7',
  '2010',
  'BURGLARY',
  'FELONY',
  'D',
  '66',
  'BROOKLYN',
  'N.Y. POLICE DEPT',
  '987478',
  '166141',
  '"(40.6227027620001',
  ' -73.9883732929999)"'],
 ['2',
  'c6245d4d',
  '12/14/1968 12:20:00 AM',
  'Saturday',
  'Dec',
  '14',
  '1968',
  '0',
  '12',
  '14',
  '2008',
  'GRAND LARCENY',
  'FELONY',
  'G',
  '28',
  'MANHATTAN',
  'N.Y. POLICE DEPT',
  '996470',
  '232106',
  '"(40.8037530600001',
  ' -73.955861904)"'],
 ['3',
  '716dbc6f',
  '10/30/1970 03:30:00 PM',
  'Friday',
  'Oct',
  '30',
  '1970',
  '15',
  '10',
  '31',
  '2008',
  'BURGLARY',
  'FELONY',
  'H',
  '84',
  'BROOKLYN',
  'N.Y. POLICE DEPT',
  '986508',
  '190249',
  '"(40.688874254',
  ' -73.9918594329999)"'],
 ['4',
  '638cd7b7',
  '07/18/1972 11:00:00 PM',
  'Tuesday',
  'Jul',
  '18',
  '1972',
  '23',
  '7',
  '19',
  '2012',
  'GRAND LARCENY OF MOTOR VEHICLE',
  'FELONY',
  'F',
  '73

In [9]:
import csv
from io import StringIO
from collections import namedtuple

In [10]:
print(header)

OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1


In [11]:
fields = header.replace(" ", "_").replace("/","_").split(",")

In [12]:
print(fields)

['OBJECTID', 'Identifier', 'Occurrence_Date', 'Day_of_Week', 'Occurrence_Month', 'Occurrence_Day', 'Occurrence_Year', 'Occurrence_Hour', 'CompStat_Month', 'CompStat_Day', 'CompStat_Year', 'Offense', 'Offense_Classification', 'Sector', 'Precinct', 'Borough', 'Jurisdiction', 'XCoordinate', 'YCoordinate', 'Location_1']


In [13]:
crime = namedtuple('crime', fields)

In [14]:
def parse(row):
  reader = csv.reader(StringIO(row))
  row=reader.next()
  return crime(row)

In [15]:
crimes=dataWoHeader.map(parse)

In [40]:
crimes.first()

Py4JJavaError: ignored

## Identifying missing values
#### Filtering records with missing values

In [21]:
crimes.map(lambda x:x.Offense).countByValue()

In [ ]:
crimes.map(lambda x:x.Occurrence_Year).countByValue()

In [ ]:
crimesFiltered = crimes.filter(lambda x: not (x.Offense=="NA" or x.Occurrence_Year==' '))\
                       .filter(lambda x: int(x.Occurrence_Year)>=2006)

In [ ]:
crimesFiltered.map(lambda x:x.Occurrence_Year).countByValue()

### Identifying anomalies
#### Filtering records with anomalies